# Simple example of time domain beamforming

Let's imagine that we have one or more idealised point sound sources with spherical spreading impinging upon an n-element linear microphone array.

The situation is depicted in the figures below.

We have used the following helpful links in developing this example:

* [Jupyter Interactors](https://github.com/bokeh/bokeh/tree/2.3.0/examples/howto/notebook_comms/Jupyter%20Interactors.ipynb)
* [Jupyter User guide](https://docs.bokeh.org/en/latest/docs/user_guide/jupyter.html)
* [Equations for Plane Waves, Spherical Waves, and Gaussian Beams](https://onlinelibrary.wiley.com/doi/pdf/10.1002/9781118939154.app3)
* https://en.wikipedia.org/wiki/Wavelength


Note that this interact does not seem to work in JupyterLab 3 at the moment.


## Import modules

In [ ]:
from ipywidgets import interact
import numpy as np

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import gridplot
import seaborn as sns
output_notebook()

## Set up parameters

In [ ]:
# Array characteristics
num_elements = 3
spacing = 0.1 # metres
x_location = 0.5 # metres
Arr = []
for k in range(num_elements):
    Arr.append([x_location,k*spacing])
Arr = np.array(Arr)
col = sns.color_palette(None, 3).as_hex()


In [ ]:
# Source characteristics
num_sources = 1
Src = {}
Src['position'] = []
Src['amplitude'] = []
Src['wavelength'] = []
for n in range(num_sources):
    Src['position'].append([-0.5,n-0.25]) # Source position
    Src['amplitude'].append(1) # Source amplitude
    Src['wavelength'].append(0.1) # Source wavelength

#Src['amplitude'][1] = 0

In [ ]:
sample_rate = 96000
sample_length = 0.001 # seconds
t_0 = 0
π = np.pi
c = 340 # m/s (Speed of sound)

N = 500
x_min = -1
x_max = 1
x = np.linspace(x_min, x_max, N)
y = np.linspace(x_min, x_max, N)
t = np.arange(t_0, t_0+sample_length, 1/sample_rate)
xx, yy = np.meshgrid(x, y)

Z = 0*xx
for n in range(num_sources):
    [s_x, s_y] = Src['position'][n] #location of source
    A = Src['amplitude'][n]
    λ = Src['wavelength'][n] # metres
    f = c/λ # Hz

    print(f'Source {n+1}: A = {A}, f = {f} Hz, λ = {λ} m , T = {1/f:0.2e} s') 

    rr = np.sqrt((xx-s_x)*(xx-s_x) + (yy-s_y)*(yy-s_y))
    
    #To make the plot look nicer, we can blank out the values within the
    #near field (within two wavelengths of the source)
    #rr = rr*(rr>2*λ)
    
    Z = Z + A/rr*np.sin(2*π/λ*(rr - c*t_0))

    p = figure(tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")],
              )
    p.xaxis.axis_label = p.yaxis.axis_label = 'Distance (m)'
    p.x_range.range_padding = p.y_range.range_padding = 0

    # must give a vector of image data for image parameter
    p.image(image=[Z],
            x=x_min, y=x_min,
            dw=(x_max-x_min), dh=(x_max-x_min), 
            palette="Spectral11", level="image")

for k in range(num_elements):
    p.circle(Arr[k,0],Arr[k,1],
             size=10,
             fill_color=col[k],
             legend_label=f'Array Element {k}',
             )

p.grid.grid_line_width = 0.5
p.legend.click_policy="hide"

q = figure()
q.xaxis.axis_label = 'Time (s)'
q.yaxis.axis_label = 'Pressure'

Y = []
for k in range(num_elements):
    F = 0*t
    for n in range(num_sources):
        [s_x, s_y] = Src['position'][n] #location of source
        A = Src['amplitude'][n]
        λ = Src['wavelength'][n] # metres
        f = c/λ # Hz
    
        r = np.sqrt((Arr[k,0]-s_x)*(Arr[k,0]-s_x) + 
                    (Arr[k,1]-s_y)*(Arr[k,1]-s_y))
        F = F + A/r*np.sin(2*π/λ*(r - c*t))
    Y.append(F)
    q.line(t,F,color=col[k],legend_label=f'Array Element {k}')

q.legend.click_policy="hide"

show(gridplot([p, q], ncols=2), notebook_handle=True)

In [ ]:
def update(f, w=1, A=1, phi=0):
    if   f == "sin": func = np.sin
    elif f == "cos": func = np.cos
    r.data_source.data['y'] = A * func(w * x + phi)
    push_notebook()

In [ ]:
show(p, notebook_handle=True)

In [ ]:
interact(update, f=["sin", "cos"], w=(0,50), A=(1,10), phi=(0, 20, 0.1))

In [ ]:
p.image?